<a href="https://colab.research.google.com/github/dave502/NLP/blob/main/lesson_03/nlp_hw_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install corus
!pip install annoy
!pip install pymorphy2
!pip install stop_words

In [50]:
from corus import load_ods_gazeta
from google.colab import drive
import annoy
from gensim.models import Word2Vec, FastText
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import string
from tqdm.notebook import tqdm
import pickle
import numpy as np

In [3]:
data_path = '/drive/My Drive/MLData/'
data_file = 'gazeta.csv.gz'

In [4]:
drive.mount('/drive')

Mounted at /drive


In [13]:
# функция предобработки текста
def preprocess_text(text, morpher, stopwords, punctuation):

  tokens = ''.join(token for token in text.strip() if token not in punctuation).split()
  tokens = [morpher.parse(token.lower())[0].normal_form for token in tokens]
  tokens = [token for token in tokens if token not in stopwords and token != ""]
  return tokens  

In [14]:
# даные для предобработки текста
morpher = MorphAnalyzer()
stopwords = set(get_stop_words("ru"))
punctuation = set(string.punctuation)

In [15]:
# загрузка даных
records = load_ods_gazeta(data_path+data_file)
next(records)

NewsRecord(
    timestamp=datetime.datetime(2008, 11, 21, 15, 19, 14),
    url='https://www.gazeta.ru/news/business/2008/11/21/n_1298950.shtml',
    edition=None,
    topics='Бизнес',
    authors=None,
    title='Госдума сокращает срок действия ставки экспортных пошлин на нефть',
    text='Госдума приняла сегодня в первом чтении и сразу в целом поправки в закон «О таможенном тарифе», сокращающие срок действия ставки экспортных пошлин на нефть с 2-х до 1-го месяца.nnДля установления средних цен на нефть марки Urals и расчета экспортных пошлин правительство России в течение двух месяцев проводит мониторинг на международных рынках нефтяного сырья (средиземноморском и роттердамском), экспортные пошлины на нефть устанавливаются также раз в два месяца.nnСокращение на месяц периода мониторинга (с 15-го числа каждого календарного месяца по 14-е число следующего месяца) и соответственно срока действия ставок экспортных пошлин «позволит более оперативно реагировать на изменения экономической сит

In [16]:
# предобработка и запись новых данных в список
news = []
counter = 0
for record in tqdm(records):
  news.append(preprocess_text(record.text, morpher, stopwords, punctuation))
  counter += 1
  if counter > 100000: break


0it [00:00, ?it/s]

In [159]:
print(news[0])

['японский', 'учёный', 'физический', 'центр', 'riken', 'заявлять', 'удаться', 'синтезировать', 'атом', '113го', 'элемент', 'таблица', 'менделеев', 'элемент', 'впервые', 'получить', '2003', 'российский', 'американский', 'физик', 'эксперимент', 'синтез', '115го', 'элемент', 'открытие', 'признать', 'международный', 'союз', 'теоретический', 'прикладной', 'химия', 'сообщаетnриа', '«новости»nnть', 'природа', 'существовать', 'элемент', 'атомный', 'номер', 'число', 'протон', 'ядро', 'атом', 'большой', '92', 'тяжёлый', 'уран', 'тяжёлый', 'элемент', 'например', 'плутоний', 'нарабатываться', 'атомный', 'реактор', 'элемент', 'тяжёлый', '100го', 'фермий', 'получать', 'ускоритель', 'путём', 'бомбардировка', 'мишень', 'тяжёлый', 'ион', 'слияние', 'ядро', 'мишень', '«снаряда»', 'возникать', 'ядро', 'новый', 'элементаnnгруппа', 'учёный', 'руководство', 'косука', 'морить', 'статья', 'опубликовать', 'journal', 'of', 'physical', 'society', 'of', 'japan', 'описывать', 'результат', 'многолетний', 'экспериме

In [18]:
with open(data_path+'news.pkl', 'wb') as f:
  pickle.dump(news, f)

In [ ]:
%%time
# обучение модели
modelFT = FastText(sentences=news, min_count=3, window=5, workers=8, size=300)

In [ ]:
# повторная загрузка данных для кластеризатора
records = load_ods_gazeta(data_path+data_file)
# инициализация кластеризатора
ft_index = annoy.AnnoyIndex(300 ,'angular')

index_map = {}
counter = 0
# для каждой новости
for record in tqdm(records):
    n_ft = 0
    # запись новости в словарь
    index_map[counter] = record.text

    # инициализация вектора новости нолями
    vector_ft = np.zeros(300)

    # для каждого слова новости получаем вектор из модели и суммируем все вектора в один
    for word in record.text.split():
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word]
            n_ft += 1
    if n_ft > 0:
        vector_ft = vector_ft / n_ft # нормализация вектора 
    # добавляем полученный вектор в кластеризатор 
    ft_index.add_item(counter, vector_ft)
    
    counter += 1

    if counter > 100000:
        break

In [ ]:
    ft_index.build(1000)

In [ ]:
# функция получения новости по запросу
def get_response(query_text, index, model, index_map, n_results):
    # предобработка запроса
    query = preprocess_text(query_text, morpher, stopwords, punctuation)
    # инициализация и формирование ветора запроса
    vector = np.zeros(300)
    norm = 0
    for word in query:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    # поиск индексов близких новостей кластеризатором
    news_indexes = index.get_nns_by_vector(vector, n_results, )
    # вывод новостей по индексам
    return [index_map[i] for i in news_indexes]

In [134]:
text = 'новости спорта в россии'

In [136]:
get_response(text, ft_index, modelFT, index_map, 5)

['Согласно постановлению правительства России, 17 лауреатам в области физической культуры и спорта выплатят денежное поощрение в размере 500 тыс. руб. в качестве поощрения. Об этом сообщается на сайтеnправительства Россииn.nЛауреаты определятся в десяти национальных номинациях в области физкультуры и спорта: «Гордость России», «Надежда России», «За служение спорту», «Спортивная солидарность», «Регион России», «Эпоха в спорте», «Преодоление», «Спортивный Парнас», «Спортивный объект в России», «За вклад в развитие физической культуры и спорта в сельской местности».nnДействие указанного постановления продлено до 2017 года в целях создания стимула для спортсменов, тренеров, специалистов в области физической культуры и спорта.nОзнакомиться с другими материалами, новостями и статистикой можно наnстраницеnлетних видов спорта.',
 'В Екатеринбурге состоялась торжественная церемония открытия футбольного манежа «Урал», сообщаетn«ФЦП-Пресс»n. В ней приняли участие министр спорта РФ Виталий Мутко, 